In [1]:
from IPython.display import display, HTML
from IPython.core.magic import register_cell_magic
from ipywidgets import interact, interactive, Layout, fixed, interact_manual
import ipywidgets as widgets



import handcalcs.render
from handcalcs.decorator import handcalc

from math import sqrt, sin, tan, cos, radians
import time

import collections

from pint import UnitRegistry, set_application_registry
ureg = UnitRegistry()
set_application_registry(ureg)

import pint
u_pint = pint.UnitRegistry()
u_pint.default_format = '~'
#u_pint.load_definitions('my_def.txt') # doctest: +SKIP
Q_pint = u_pint.Quantity

ModuleNotFoundError: No module named 'handcalcs'

In [4]:
CSS = """
.widget-vbox {
    flex-direction: row;
    display: block;
}

.widget-inline-hbox {
    display: initial;
}

.jp-OutputArea-output.jp-RenderedLatex .MathJax_Display {
    text-align: center !important;
}

.jp-OutputArea-output {
    overflow: visible !important;
}
"""

##HTML('<style>{}</style>'.format(CSS)) 
## uncomment the above to centre

In [5]:
box_layout = Layout(width='200px',
                    height='',
                    flex_flow='row',
                    display='vertical-align')

In [6]:
@register_cell_magic('handle')
def handle(line, cell):
    try:
        exec(cell)
    except Exception as e:
        display("failed")
        #raise # if you want the full trace-back in the notebook

In [7]:
def fltxt(cons: float):
    fWidg = widgets.FloatText(cons, layout=box_layout)
    return fWidg

def create_new_cell(contents):
    from IPython.core.getipython import get_ipython
    shell = get_ipython()

    payload = dict(
        source='set_next_input',
        text=contents,
        replace=False,
    )
    shell.payload_manager.write_payload(payload, single=False)

# Variables

In [15]:
Mass = fltxt(5.972*10**24)
Gravity = fltxt(6.67408*10**-11)
SpeedofLight = fltxt(299792458)

_Mass=[('Earth',5.972*10**24), ('Sun',1.989*10**30), ('Mars',6.39*10**23)]

u_Mass = u_pint.kg
u_Gravity = (u_pint.m**3)*(u_pint.kg**-1)*(u_pint.s**-2)
u_SpeedofLight = u_pint.meter/u_pint.s

Form = collections.namedtuple('Form',['name','symbol','eq'])

# Functions

In [16]:
@interact(M = _Mass,
          G = Gravity,
          c = SpeedofLight)
@handcalc(override='long', jupyter_display=True)
def f_Rs(M: float,G: float,c: float):
    R_s = (2*G*M)/(c**2)
    return R_s

interactive(children=(Dropdown(description='M', options=(('Earth', 5.972e+24), ('Sun', 1.9890000000000002e+30)…

In [10]:
def g(Mass: float,Gravity: float,SpeedofLight: float):
    f_Rs(Mass*u_Mass,Gravity*u_Gravity,SpeedofLight*u_SpeedofLight)

In [11]:
@interact(Mass = _Mass,
          Gravity = Gravity,
          SpeedofLight = SpeedofLight)
def h(Mass: float,Gravity: float,SpeedofLight: float):
    f_Rs(Mass*u_Mass,Gravity*u_Gravity,SpeedofLight*u_SpeedofLight)

interactive(children=(Dropdown(description='Mass', options=(('Earth', 5.972e+24), ('Sun', 1.9890000000000002e+…

In [12]:
interactive(g, Mass =_Mass, Gravity = Gravity,SpeedofLight = SpeedofLight)

interactive(children=(Dropdown(description='Mass', options=(('Earth', 5.972e+24), ('Sun', 1.9890000000000002e+…

# Calls

In [13]:
##TO DO
# Add Pint units to newFormula
# Fix inter_Disp
# If constant, dont return floating text?
#or add a button which can show and make disapear the constants
# If no constant found return input to add to constant file
# replace ** as ^ just for the input and replace it in the function back to **

In [14]:
def create_Params(equation):
    eq_Chars = equation.eq
    inter_Params = ""
    handc_Params = ""
    var_Params = ""
    flt_Params = ""
    unit_Params = ""
    aa = eq_Chars
    excluded = '/ * + - 1 2 3 4 5 6 7 8 9 0 ( ) ^'
    exc2 = '/ * + - ( )'
    for x in eq_Chars:
        if excluded.split().count(x) == 0 : #any spaces in the equation will break this
            inter_Params += (x + " = fltxt("+ x +"), ")
            handc_Params += (x + " :float, ")
            var_Params += x + ", "
            flt_Params += x + ' = ' + x + "'.flttxt', "
            unit_Params += x + ' * ' + x + "'.unit',"
    return inter_Params, handc_Params, var_Params, flt_Params, unit_Params


    #for x in exc2:
        #aa = aa.replace(x," ")
        #print(aa)
   #Q_pint(aa)
#create_Params(Form('Schwarszchild-Radius','R_s', '(2*G*M)/c**2'))

In [15]:
def newFormula(For):
    params = create_Params(For)
    eq_replace = For.eq.replace("^","**")
    form_Cell = "{}\n{}\n    Sci_Notation_{} = {}\n\n{}\n{}\n    {} = {}\n    Reduced_{} = {}.to_compact(){}"
    pass
    param_Disp = "@handcalc(override='params', jupyter_display=True)"
    pfunc_Disp = "def " + "p_" + For.symbol +"(" + params[1] + "):"
    pass
    jup_Disp = "@handcalc(override='long', jupyter_display=True)"
    def_Disp = "def " + "f_" + For.symbol +"(" + params[1] + "):"
    pass
    inter_Disp = "\n\n@interact(" + params[3] + ")\n"
    interDef_Disp = "def " + "c_" + For.symbol + "(" + params[2] + "):\n"
    call_Cell = inter_Disp + interDef_Disp + "    f_" + For.symbol + "(" + params[4] + ")\n" + "    p_" + For.symbol + "(" + params[2] + ")"
    pass
    create_new_cell(form_Cell.format(param_Disp, pfunc_Disp,For.symbol, eq_replace, jup_Disp, def_Disp, For.symbol, eq_replace, For.symbol,For.symbol, call_Cell))

In [16]:
newFormula(Form('Schwarszchild-Radius','R_s', '(2*G*M)/c^2'))

In [ ]:
@handcalc(override='params', jupyter_display=True)
def p_R_s(G :float, M :float, c :float, ):
    Sci_Notation_R_s = (2*G*M)/c**2

@handcalc(override='long', jupyter_display=True)
def f_R_s(G :float, M :float, c :float, ):
    R_s = (2*G*M)/c**2
    Reduced_R_s = R_s.to_compact()

@interact(G = G'.flttxt', M = M'.flttxt', c = c'.flttxt', )
def c_R_s(G, M, c, ):
    f_R_s(G * G'.unit',M * M'.unit',c * c'.unit',)
    p_R_s(G, M, c, )

In [17]:
@handcalc(override='params', jupyter_display=True)
def p_R_s(G :float, M :float, c :float, ):
    Sci_Notation_R_s = (2*G*M)/c**2

@handcalc(override='long', jupyter_display=True)
def f_R_s(G :float, M :float, c :float, ):
    R_s = (2*G*M)/c**2
    Reduced_R_s = R_s.to_compact()

@interact(G = G'.flttxt', M = M'.flttxt', c = c'.flttxt', )
def c_R_s(G, M, c, ):
    f_R_s(G * G'.unit',M * M'.unit',c * c'.unit',)
    p_R_s(G, M, c, )

SyntaxError: invalid syntax (<ipython-input-17-26395a87d8f2>, line 10)

In [18]:
@handcalc(override='params', jupyter_display=True)
def p_R_s(G :float, M :float, c :float, ):
    Sci_Notation_R_s = (2*G*M)/c**2

@handcalc(override='long', jupyter_display=True)
def f_R_s(G :float, M :float, c :float, ):
    R_s = (2*G*M)/c**2
    Reduced_R_s = R_s.to_compact()

@interact(G = G'.flttxt', M = M'.flttxt', c = c'.flttxt', )
def c_R_s(G, M, c, ):
    f_R_s(G * G'.unit',M * M'.unit',c * c'.unit',)
    p_R_s(G, M, c, )

SyntaxError: invalid syntax (<ipython-input-18-26395a87d8f2>, line 10)

In [19]:
@handcalc(override='params', jupyter_display=True)
def p_R_s(G :float, M :float, c :float, ):
    Sci_Notation_R_s = (2*G*M)/c**2

@handcalc(override='long', jupyter_display=True)
def f_R_s(G :float, M :float, c :float, ):
    R_s = (2*G*M)/c**2
    Reduced_R_s = R_s.to_compact()

@interact(G = G'.flttxt', M = M'.flttxt', c = c'.flttxt', )
def c_R_s(G, M, c, ):
    f_R_s(G * G'.unit',M * M'.unit',c * c'.unit',)
    p_R_s(G, M, c, )

SyntaxError: invalid syntax (<ipython-input-19-26395a87d8f2>, line 10)

In [35]:
@handcalc(override='params', jupyter_display=True)
def p_R_s(G :float, M :float, c :float, ):
    Sci_Notation_R_s = (2*G*M)/c**2

@handcalc(override='long', jupyter_display=True)
def f_R_s(G :float, M :float, c :float, ):
    R_s = (2*G*M)/c**2
    Reduced_R_s = R_s.to_compact()

@interact(G = widgets.Text("2.75"), M = 2000, c = 666, )
def c_R_s(G, M, c, ):
    f_R_s(u_pint.formatter(Q_pint(G) ,M,c)
    p_R_s(Q_pint(G).magnitude, M, c, )

SyntaxError: invalid syntax (<ipython-input-35-92a38cd5444e>, line 13)

In [36]:
dir(Q_pint("2.75kg"))

['T',
 'UnitsContainer',
 '_Quantity__handling',
 '_Quantity__ito_if_needed',
 '_Quantity__used',
 '_REGISTRY',
 '__abs__',
 '__add__',
 '__array__',
 '__array_function__',
 '__array_priority__',
 '__array_ufunc__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__complex__',
 '__copy__',
 '__dask_graph__',
 '__dask_keys__',
 '__dask_optimize__',
 '__dask_postcompute__',
 '__dask_postpersist__',
 '__dask_scheduler__',
 '__dask_tokenize__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__po

In [5]:
f = open("my_def.txt", "a")
f.write("Now the file has more content!")
f.close()

#open and read the file after the appending:
f = open("my_def.txt", "r")
print(f.read()) 

Now the file has more content!


In [6]:
open('my_def.txt', 'w').close()
print(f.read()) 

In [ ]:
dir(u)